In [ ]:
from __future__ import print_function
import gzip
import os
import numpy as np
import random

In [ ]:
words = open("text8.bz2").read().decode("bz2").split()

print('Data size', len(words))

In [ ]:
# Step 2: Build the dictionary and replace rare words with UNK token.
import collections
vocabulary_size = 50000

def build_dataset(words):
    count = [('UNK', 0)]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = {word: i for i, (word, _) in enumerate(count)}
    data = [dictionary.get(word, 0) for word in words]
    
    reverse_dictionary = {v: k for k, v in dictionary.iteritems()}
    unk_count = len(words)-sum(w[1] for w in count)
    count[0] = ('UNK', unk_count)
    
    return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

data_index = 0

In [ ]:
# Step 4: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size / num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], '->', labels[i, 0])
    print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])

In [ ]:
import tensorflow as tf
import math

In [ ]:
def gpu_node(n):
  # print(n.type)
  if n.type == "MatMUl":
    return "/gpu:0"
  else:
    return "/cpu:0"

In [ ]:
# Step 5: Build and train a skip-gram model.

batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 3       # How many words to consider left and right.
num_skips = 4         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(xrange(valid_window), valid_size))
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():
  with graph.device(gpu_node):
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Construct the variables.
    embeddings = tf.Variable(
      tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    nce_weights = tf.Variable(
      tf.truncated_normal([vocabulary_size, embedding_size],
                          stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
      tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                     num_sampled, vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

In [ ]:
session = tf.InteractiveSession(graph=graph)

In [ ]:
session.run(tf.initialize_all_variables())

In [ ]:
# Step 6: Begin training
num_steps = 2000001

with session.as_default():
  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 50000 == 0:
      if step > 0:
        average_loss = average_loss / 50000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 500000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  result_embeddings = embeddings.eval()
  final_embeddings = normalized_embeddings.eval()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  #in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i,:]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

try:
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only,:])
  labels = dictionary.keys()[:plot_only]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print("Please install sklearn and matplotlib to visualize embeddings.")

In [ ]:
def find_sim(v, num=10):
    #return np.dot(final_embeddings, v).argsort()[-num:][::-1]
    return [reverse_dictionary[x] for x in  np.dot(final_embeddings, v).argsort()[-num:][::-1]]
def w2v(w):
    return result_embeddings[dictionary.get(w, 0)] 
print(find_sim(w2v('four')))

In [ ]:
target = find_sim(w2v('king')-w2v('man')+w2v('woman'))
print(target)

In [ ]:
v1 = w2v('king')-w2v('man')+w2v('woman')
v2 = w2v('queen')
np.dot(v1, v2) /np.linalg.norm(v1)/np.linalg.norm(v2) 

In [ ]:
v1 = w2v('king')-w2v('man')+w2v('woman')
v2 = w2v(target[0])
np.dot(v1, v2) /np.linalg.norm(v1)/np.linalg.norm(v2) 

In [ ]:
np.linalg.norm(v1), np.linalg.norm(v2) 

In [ ]:
find_sim(w2v('eat')-w2v('food'))


In [ ]:
with graph.as_default():
    saver = tf.train.Saver()

In [ ]:
save_path = saver.save(session, "save/word2vec.ckpt")
print("Model saved in file: ", save_path)

In [ ]:
saver.restore(session, "save/word2vec.ckpt")